In [11]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
from app.database import SessionLocal, Base, engine
from app.models import *
from app import models
from app.utils import hash_password
from app.db_utils import (
    reset_db,
    drop_table, 
    to_df, 
    get_table_names, 
    count_records, 
    backup_table,
    get_table_schema,
    get_session,
    execute_raw_query
)
import random
import uuid
from typing import List, Dict, Set, Tuple
from devseed import gather_unique_cf_handles

import numpy as np
from faker import Faker
from app.models import (
    User,
    Group,
    GroupMembership,
    Report,
    Contest,
    ContestParticipation,
    Role,
)
from sqlalchemy import func

def random_subset(n, k):
    return random.sample([i for i in range(n)], k)

def pick_random_subset(population: list, k: int) -> list:
    """k distinct elements from population (k may be 0)."""
    if k == 0:
        return []
    return random.sample(population, k)

def get_mock_contest_standings(cid, include_users=None):
    if include_users is None:
        include_users = []
    res = {
        'contest': {
            'id': cid,
            'name': f'Codeforces Round {cid}',
            'type': 'CF',
            'phase': 'FINISHED',
            'frozen': False,
            'durationSeconds': 7200,
            'startTimeSeconds': 1746110100,
            'relativeTimeSeconds': 267545
        },
        'standings': [
        ]
    }
    num_participants = 8000
    standings = [
        {'handle': 'random-user'+str(i+1), 'rank': (i+1), 'points': 9754 - i, 'penalty': 0}
        for i in range(num_participants)
    ]
    res['standings'] = standings
    udx = pick_random_subset([i for i in range(num_participants)], len(include_users))

    for i in range(len(udx)):
        res['standings'][udx[i]]['handle'] = include_users[i]
    
    return list(set(res))

In [12]:
cf_handles = gather_unique_cf_handles([2102, 2101, 2109, 2107])


» cf api → contest 2102

» cf api → contest 2101

» cf api → contest 2109

» cf api → contest 2107

» cf handles gathered ➜ 5333 unique


In [10]:
SEED = 88
random.seed(SEED)
np.random.seed(SEED)
Faker.seed(SEED)
faker = Faker()
DEFAULT_PASS = "devpass"

In [ ]:
admin_users = [
    User(
        user_id="shrey",
        role=Role.admin,
        cf_handle="misaki",
        email_id="talmudlover@gmail.com",
        atcoder_handle=None,
        codechef_handle=None,
        twitter_handle=None,
        trusted_score=88,
        hashed_password=hash_password(DEFAULT_PASS),
    ),
    User(
        user_id="ani",
        role=Role.admin,
        cf_handle="roomTemperatureIQ",
        email_id="ecwcejhvchv@gmail.com",
        atcoder_handle=None,
        codechef_handle=None,
        twitter_handle=None,
        trusted_score=88,
        hashed_password=hash_password(DEFAULT_PASS),
    ),
]

users = admin_users

for handle in cf_handles:
    uid = 'testUser' + str(len(users)-1)
    users.append(
        User(
            user_id=uid,
            role=Role.user,
            cf_handle=uid + '_cf',
            atcoder_handle=None if random.random() < 0.5 else uid + "_ac",
            codechef_handle=None if random.random() < 0.7 else uid + "_cc",
            twitter_handle=None if random.random() < 0.6 else uid + "_tw",
            trusted_score=random.randint(0, 100),
            email_id=str(uid) + '@gmail.com',
            hashed_password=hash_password(DEFAULT_PASS),
        )
    )